In [2]:
from abc import ABCMeta
import logging

import numpy
import rasterio
import rasterio.features
import affine
import xarray

# from . import ingest_base

In [9]:
    def read_data_from_gdal(input_file, variable_name, expstr=None, band=1,
                            resampling="nearest", **kwargs):
        """
        Function to load GDAL file into an xarray

        Uses xarray.open_rasterio function.

        Takes an array with x coordinates and y coordinates and uses
        these to resample / re-grid data if required

        :param str input_file: input file in any format GDAL can read.
        :param str variable_name: name to use for variable within output xarray,.
                                  Variable name must not contain non-alphanumeric
                                  characters.
        :param str expstr: expression to evaluate using input band. Can be used to
                           apply a threshold or scaling factor. Must take form of
                           'band > 100'
        :param int band: band to use from raster (default=1)
        :param str resampling: method to use when resampling data (default=nearest)
        :param kwargs: any other input parameters.
        :return: xarray Dataset containing variable(s) with coordinates self.object_ids.
        :rtype: xarray.Dataset
        """
#         # Check the output variable name
#         if not check_variable_name(variable_name):
#             raise Exception("Input variable name is inappropriate, see log.")

#         try:
#             resampling_method = getattr(rasterio.enums.Resampling, resampling)
#         except AttributeError:
#             raise AttributeError("Method for resampling ({}) not supported. "
#                                  "Valid options are: {}".format(resampling,
#                                  ",".join([str(i) for i in  rasterio.enums.Resampling])))

        with rasterio.open(input_file) as ds:
            # Get pze and check against target.
            # Print info to log file if they are different.
            transform = ds.get_transform()
            if target_pixel_size_x != transform[1] or \
              target_pixel_size_y != transform[5]:
                logging.info("Interpolating {} from pixel size of: {},{}"
                                " to {},{} using {}.".format(input_file,
                                                   transform[1], transform[5],
                                                   target_pixel_size_x,
                                                   target_pixel_size_y,
                                                   resampling))

            # Open band (don't load)
            band = rasterio.band(ds, int(band))
            # Set up output array for data
            out_array = numpy.empty((target_y_coords.size,
                                     target_x_coords.size),
                                     dtype=band.dtype)

            # Perform windowed read to get data for target array
            # Resample as data is being read.
            rasterio.warp.reproject(source=band,
                                    destination=out_array,
                                    dst_crs=target_crs,
                                    dst_transform=target_transform,
                                    resampling=resampling_method)

        # If using an output time then need to add an extra dimension for it.
        if output_time is not None:
            out_array = out_array.reshape([1, out_array.shape[0], out_array.shape[1]])

        # Convert to xarray
        variable_xarray = xarray.Dataset({variable_name: (output_dims,
                                                          out_array)},
                                         coords=output_coords)

        # Check matches existing grid
        resample_regrid(variable_xarray)

        if expstr is not None:
            # Use 'band' for expressions as more intuitive but dataset is named
            # using variable name so replace in expression before
            # running.
            variable_xarray = xarray_maths(variable_xarray, variable_name,
                                                [variable_name],
                                                expstr.replace("band", variable_name))
        return variable_xarray

In [10]:
read_data_from_gdal('/opt/ls8_mad_250.tif', 'variable_name', expstr=None, band=1, resampling="nearest")

RasterioIOError: /opt/ls8_mad_250.tif: No such file or directory